# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 - Creating the necessary Dataframe

### Retreiving Data

In [1]:
# Let's import the necessary dependencies

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
# Let's scrape the wikipedia webpage and retreive the necessary data
wiki_path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(wiki_path)

In [3]:
# Let's check how many tables were read into the variable data from the wikipedia webpage
print(len(data))

3


In [4]:
# It looks like we have retreive 3 tables from the webpage. Let's examine them and select the one we need.
data[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


It seems that this is the table we are looking for, so we'll skip the rest.

### Data Pre-processing

In [5]:
# Let's process the above data for a cleaner dataframe to work with
Toronto_data = data[0][data[0]['Borough'] != 'Not assigned'].reset_index(drop = True) # Removes rows with not assigned Borough.
Toronto_data.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
Toronto_data.shape

(103, 3)

## Part 2 - Getting the latitude and the longitude coordinates of each neighborhood

Double-click __here__ for Method 1.
<!-- 
I Could not Use this method because it kept runnig forever, however below is the code I used.

! pip install geocoder
-->

<!--
import geocoder # import geocoder

for postal_code in Toronto_data['Postal Code']:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    Toronto_data = Toronto_data.append({'Latitude': latitude, 'Longitude': longitude})

Toronto_data.head()
-->

In [7]:
# I used this method instead

csv_path = 'http://cocl.us/Geospatial_data'
df_latlon = pd.read_csv(csv_path)
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_latlon.shape

(103, 3)

In [9]:
# Let's merge the two dataframes based on the Postal Code Column
master_df = pd.merge(Toronto_data, df_latlon, on = 'Postal Code')
master_df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [10]:
master_df.shape

(103, 5)

## Part 3 - Explore and cluster the neighborhoods in Toronto

In [11]:
print('There are', len(master_df['Borough'].unique()), 'Boroughs in Toronto which are:', master_df['Borough'].unique())

There are 10 Boroughs in Toronto which are: ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


Let's get the geographical coordinates of Toronto.

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>t_explorer</em>, as shown below.

In [13]:
!pip install geopy
#!conda install -c conda-forge geopy --yes # install the package
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [14]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

     |████████████████████████████████| 102kB 11.7MB/s ta 0:00:01


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(master_df['Latitude'], master_df['Longitude'], master_df['Borough'], master_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's explore and cluster the Central Toronto Borough.

In [40]:
Central_Toronto = master_df[master_df['Borough'] == 'Central Toronto'].reset_index(drop=True)
Central_Toronto = Central_Toronto.rename(columns = {'Neighbourhood':'Neighborhood'})
Central_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


Let's get the geographical coordinates of Manhattan.

In [41]:
address2 = 'Central Toronto, TO'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address2)
latitude2 = location.latitude
longitude2 = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [42]:
# create map of Central Toronto using latitude and longitude values
map_Central_toronto = folium.Map(location=[latitude2, longitude2], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Central_Toronto['Latitude'], Central_Toronto['Longitude'], Central_Toronto['Borough'], Central_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Central_toronto)  
    
map_Central_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [43]:
CLIENT_ID = 'WBR0WSJCSHUZIU2BR1BOD32R45AX24NPQZBFGJWHVDGWM5CH' # your Foursquare ID
CLIENT_SECRET = '0SHD2S5ZT4AMTTSDBABRZU0DLFGBKGBILIFVO11BMPFCSXE1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WBR0WSJCSHUZIU2BR1BOD32R45AX24NPQZBFGJWHVDGWM5CH
CLIENT_SECRET:0SHD2S5ZT4AMTTSDBABRZU0DLFGBKGBILIFVO11BMPFCSXE1


#### Let's create a function explores all the neighborhoods in Central Toronto. We will treat neighborhoods with the same postal code as 1 neighborhood.

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
Central_Toronto_Venues = getNearbyVenues(names=Central_Toronto['Neighborhood'],
                                   latitudes=Central_Toronto['Latitude'],
                                   longitudes=Central_Toronto['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


#### Let's check the size of the resulting dataframe

In [46]:
print(Central_Toronto_Venues.shape)
Central_Toronto_Venues.head(10)

(113, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
5,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue
6,Roselawn,43.711695,-79.416936,THA Home Additions Toronto,43.708369,-79.420434,Home Service
7,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
8,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
9,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [47]:
Central_Toronto_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",2,2,2,2,2,2
"North Toronto West, Lawrence Park",20,20,20,20,20,20
Roselawn,3,3,3,3,3,3
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",21,21,21,21,21,21


#### Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(Central_Toronto_Venues['Venue Category'].unique())))

There are 64 uniques categories.


#### Let's Analyze Each Neighborhood

In [49]:
# one hot encoding
C_Toronto_onehot = pd.get_dummies(Central_Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
C_Toronto_onehot['Neighborhood'] = Central_Toronto_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [C_Toronto_onehot.columns[-1]] + list(C_Toronto_onehot.columns[:-1])
C_Toronto_onehot = C_Toronto_onehot[fixed_columns]

C_Toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [50]:
C_Toronto_onehot.shape

(113, 65)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
C_Toronto_grouped = C_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
C_Toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0000,0.000000,0.0000,0.0000,0.000000,0.029412,0.000000,0.00,0.058824,...,0.0000,0.058824,0.00,0.029412,0.029412,0.029412,0.00,0.000000,0.0000,0.00
1,Davisville North,0.0000,0.000000,0.0000,0.0000,0.111111,0.000000,0.000000,0.00,0.000000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00
2,"Forest Hill North & West, Forest Hill Road Park",0.0000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.0000,0.250000,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.0000,0.00
3,Lawrence Park,0.0000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.25,0.000000,...,0.0000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00
4,"Moore Park, Summerhill East",0.0000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.50,0.000000,0.0000,0.00
5,"North Toronto West, Lawrence Park",0.0000,0.000000,0.0500,0.0000,0.000000,0.000000,0.000000,0.00,0.050000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.05
6,Roselawn,0.0000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.0625,0.000000,0.0625,0.0625,0.000000,0.000000,0.000000,0.00,0.000000,...,0.0625,0.062500,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0625,0.00
8,"The Annex, North Midtown, Yorkville",0.0000,0.047619,0.0000,0.0000,0.000000,0.000000,0.047619,0.00,0.142857,...,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.047619,0.0000,0.00


#### Let's confirm the new size

In [52]:
C_Toronto_grouped.shape

(9, 65)

#### Let's print each neighborhood along with the top 5 most common venues

In [53]:
num_top_venues = 5

for hood in C_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = C_Toronto_grouped[C_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0  Sandwich Place  0.09
1    Dessert Shop  0.09
2     Pizza Place  0.06
3             Gym  0.06
4            Café  0.06


----Davisville North----
                  venue  freq
0                 Hotel  0.11
1      Department Store  0.11
2  Gym / Fitness Center  0.11
3                   Gym  0.11
4                  Park  0.11


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2                 Park  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0          Swim School  0.25
1                 Park  0.25
2   Dim Sum Restaurant  0.25
3             Bus Line  0.25
4  Rental Car Location  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Restaurant   0.5
1                Trail   0.5
2  American Restaurant   0.0
3   Italian Restaurant   0.0
4        Jewelry 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = C_Toronto_grouped['Neighborhood']

for ind in np.arange(C_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(C_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Gym,Café,Discount Store,Park
1,Davisville North,Gym,Pizza Place,Gym / Fitness Center,Department Store,Sandwich Place,Park,Food & Drink Shop,Breakfast Spot,Hotel,Garden
2,"Forest Hill North & West, Forest Hill Road Park",Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Flower Shop,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant
3,Lawrence Park,Bus Line,Swim School,Park,Dim Sum Restaurant,Gourmet Shop,Gas Station,Garden,Greek Restaurant,Fried Chicken Joint,Diner
4,"Moore Park, Summerhill East",Restaurant,Trail,Food & Drink Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Fried Chicken Joint
5,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Chinese Restaurant,Gym / Fitness Center,Fast Food Restaurant,Italian Restaurant,Mexican Restaurant,Diner,Park,Pet Store
6,Roselawn,Home Service,Garden,Music Venue,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop
7,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Sports Bar,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant
8,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pub,Pharmacy,Pizza Place,Liquor Store,Donut Shop,Indian Restaurant,Middle Eastern Restaurant


#### Let's Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 2 clusters.

In [56]:
# set number of clusters
kclusters = 2

C_Toronto_grouped_clustering = C_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(C_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:9] 

array([0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Central_Toronto_merged = Central_Toronto

Central_Toronto_merged = Central_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [64]:
Central_Toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Swim School,Park,Dim Sum Restaurant,Gourmet Shop,Gas Station,Garden,Greek Restaurant,Fried Chicken Joint,Diner
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Home Service,Garden,Music Venue,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Pizza Place,Gym / Fitness Center,Department Store,Sandwich Place,Park,Food & Drink Shop,Breakfast Spot,Hotel,Garden
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Flower Shop,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Chinese Restaurant,Gym / Fitness Center,Fast Food Restaurant,Italian Restaurant,Mexican Restaurant,Diner,Park,Pet Store
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pub,Pharmacy,Pizza Place,Liquor Store,Donut Shop,Indian Restaurant,Middle Eastern Restaurant
6,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Gym,Café,Discount Store,Park
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Restaurant,Trail,Food & Drink Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Fried Chicken Joint
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,0,Pub,Coffee Shop,Sports Bar,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant


Finally, let's visualize the resulting clusters

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_Toronto_merged['Latitude'], Central_Toronto_merged['Longitude'], Central_Toronto_merged['Neighborhood'], Central_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# The End. Thank You! :)